In [1]:
import os
import git
from pathlib import Path

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
os.chdir(os.path.join(ROOT_DIR, "results", "combined_results"))
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
main_df = main_df.copy()
np.random.seed(0)

In [3]:
main_df2 = main_df.fillna("None", inplace=False)

In [4]:
failCatDF = main_df.copy()
failCatDF = failCatDF[failCatDF["failure_type"]!= "low samples"]

failCatDF["pass"] = failCatDF["failure_category"].apply(lambda x: x in ["practically_pass", "actually_pass"])
failCatDF = failCatDF[["dataset_type","dataset", "subset", "transform", "orientation", "failure_category", "pass", "beat_all_priors", "best_prior"]]
failCatDF.fillna("None", inplace=True)
failCatDF["number"] = (
    failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])["pass"]
    .transform("count")
)

failCatDF["pass_percentage"] = np.round(
    failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"])["pass"]
    .transform("mean") * 100
)

failCatDF["beat_all_priors_percentage"] = np.round(
    failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"])["beat_all_priors"]
    .transform("mean") * 100
)


failCatDF = failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"]).first().reset_index().sort_values(by=["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])
failCatDF


,dataset_type,dataset,subset,transform,orientation,failure_category,pass,beat_all_priors,best_prior,number,pass_percentage,beat_all_priors_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,practically_pass,True,1,GenGamma,6,86.0,71.0
1,medical,syntheticMRI2D,axial,wavelet,diagonal,trivial_failure,False,0,Gaussian,1,86.0,71.0
2,medical,syntheticMRI2D,axial,wavelet,horizontal,practically_pass,True,1,GenGamma,5,71.0,86.0
3,medical,syntheticMRI2D,axial,wavelet,horizontal,trivial_failure,False,0,Gaussian,2,71.0,86.0
4,medical,syntheticMRI2D,axial,wavelet,vertical,practically_pass,True,1,GenGamma,5,71.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...
89,remote sensing,spaceNet,full,wavelet,diagonal,actually_pass,True,1,GenGamma,3,62.0,91.0
90,remote sensing,spaceNet,full,wavelet,diagonal,practically_pass,True,1,GenGamma,17,62.0,91.0
91,remote sensing,spaceNet,full,wavelet,diagonal,trivial_failure,False,0,Gaussian,12,62.0,91.0
92,remote sensing,spaceNet,full,wavelet,horizVert,practically_pass,True,1,GenGamma,20,62.0,100.0


In [5]:
ordered_failcat_cols = ["actually_pass", "practically_pass", "TO DISCUSS", "interesting_failure", "trivial_failure"]
ordered_prior_cols = ["GenGamma", "Gaussian", "Laplace", "Student-T"]

# Remote Sensing

In [6]:
DATASET_TYPE = "remote sensing" 

In [7]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,remote sensing,agriVision,full,fourier,None,95.0,98.0
1,remote sensing,agriVision,full,wavelet,diagonal,100.0,100.0
2,remote sensing,agriVision,full,wavelet,horizVert,100.0,100.0
3,remote sensing,pastis,full,fourier,None,100.0,100.0
4,remote sensing,pastis,full,learned,dual_color,0.0,100.0
5,remote sensing,pastis,full,learned,misc,0.0,50.0
6,remote sensing,pastis,full,learned,multi_edge,0.0,100.0
7,remote sensing,pastis,full,learned,single_edge,0.0,100.0
8,remote sensing,pastis,full,wavelet,diagonal,100.0,100.0
9,remote sensing,pastis,full,wavelet,horizVert,100.0,96.0


In [8]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                        actually_pass  \
dataset_type   dataset    subset transform orientation                  
remote sensing agriVision full   fourier   None                   2.0   
                                 wavelet   diagonal              17.0   
                                           horizVert             11.0   
               pastis     full   fourier   None                  91.0   
                                 learned   dual_color             0.0   
                                           misc                   0.0   
                                           multi_edge             0.0   
                                           single_edge            0.0   
                                 wavelet   diagonal              57.0   
                                           horizVert             57.0   
               spaceNet   full   fourier   None                   0.0   
                                 learned   dual_color             0.0   
                                           eye                    0.0   
                                           inside_out             0.0   
                                           misc                   0.0   
                                           multi_edge             0.0   
                                           single_edge            0.0   
                                 wavelet   diagonal               9.0   
                                           horizVert              0.0   

failure_category                                        practically_pass  \
dataset_type   dataset    subset transform orientation                     
remote sensing agriVision full   fourier   None                     93.0   
                                 wavelet   diagonal                 83.0   
                                           horizVert                89.0   
               pastis     full   fourier   None                      9.0   
                                 learned   dual_color                0.0   
                                           misc                      0.0   
                                           multi_edge                0.0   
                                           single_edge               0.0   
                                 wavelet   diagonal                 43.0   
                                           horizVert                43.0   
               spaceNet   full   fourier   None                    100.0   
                                 learned   dual_color                0.0   
                                           eye                       0.0   
                                           inside_out                0.0   
                                           misc                      0.0   
                                           multi_edge                0.0   
                                           single_edge               0.0   
                                 wavelet   diagonal                 53.0   
                                           horizVert                62.0   

failure_category                                        interesting_failure  \
dataset_type   dataset    subset transform orientation                        
remote sensing agriVision full   fourier   None                         5.0   
                                 wavelet   diagonal                     0.0   
                                           horizVert                    0.0   
               pastis     full   fourier   None                         0.0   
                                 learned   dual_color                   0.0   
                                           misc                         0.0   
                                           multi_edge                   0.0   
                                           single_edge                  0.0   
                                 wavelet   diagonal                     0.0   


In [9]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                              GenGamma  Gaussian  \
dataset_type   dataset    subset transform orientation                       
remote sensing agriVision full   fourier   None             98.0       0.0   
                                 wavelet   diagonal        100.0       0.0   
                                           horizVert       100.0       0.0   
               pastis     full   fourier   None            100.0       0.0   
                                 learned   dual_color      100.0       0.0   
                                           misc             50.0      50.0   
                                           multi_edge      100.0       0.0   
                                           single_edge     100.0       0.0   
                                 wavelet   diagonal        100.0       0.0   
                                           horizVert        96.0       0.0   
               spaceNet   full   fourier   None            100.0       0.0   
                                 learned   dual_color      100.0       0.0   
                                           eye              75.0       0.0   
                                           inside_out       60.0       0.0   
                                           misc            100.0       0.0   
                                           multi_edge      100.0       0.0   
                                           single_edge     100.0       0.0   
                                 wavelet   diagonal         91.0       9.0   
                                           horizVert       100.0       0.0   

best_prior                                              Laplace  Student-T  
dataset_type   dataset    subset transform orientation                      
remote sensing agriVision full   fourier   None             0.0        2.0  
                                 wavelet   diagonal         0.0        0.0  
                                           horizVert        0.0        0.0  
               pastis     full   fourier   None             0.0        0.0  
                                 learned   dual_color       0.0        0.0  
                                           misc             0.0        0.0  
                                           multi_edge       0.0        0.0  
                                           single_edge      0.0        0.0  
                                 wavelet   diagonal         0.0        0.0  
                                           horizVert        0.0        4.0  
               spaceNet   full   fourier   None             0.0        0.0  
                                 learned   dual_color       0.0        0.0  
                                           eye             25.0        0.0  
                                           inside_out       0.0       40.0  
                                           misc             0.0        0.0  
                                           multi_edge       0.0        0.0  
                                           single_edge      0.0        0.0  
                                 wavelet   diagonal         0.0        0.0  
                                           horizVert        0.0        0.0

In [10]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                      GenGamma  \
dataset_type   dataset    subset transform failure_category                
remote sensing agriVision full   fourier   actually_pass           100.0   
                                           interesting_failure     100.0   
                                           practically_pass         98.0   
                                 wavelet   actually_pass           100.0   
                                           practically_pass        100.0   
               pastis     full   fourier   actually_pass           100.0   
                                           practically_pass        100.0   
                                 learned   None                     97.0   
                                 wavelet   actually_pass            97.0   
                                           practically_pass        100.0   
               spaceNet   full   fourier   practically_pass        100.0   
                                 learned   None                     95.0   
                                 wavelet   actually_pass           100.0   
                                           practically_pass        100.0   
                                           trivial_failure          88.0   

best_prior                                                      Gaussian  \
dataset_type   dataset    subset transform failure_category                
remote sensing agriVision full   fourier   actually_pass             0.0   
                                           interesting_failure       0.0   
                                           practically_pass          0.0   
                                 wavelet   actually_pass             0.0   
                                           practically_pass          0.0   
               pastis     full   fourier   actually_pass             0.0   
                                           practically_pass          0.0   
                                 learned   None                      3.0   
                                 wavelet   actually_pass             0.0   
                                           practically_pass          0.0   
               spaceNet   full   fourier   practically_pass          0.0   
                                 learned   None                      0.0   
                                 wavelet   actually_pass             0.0   
                                           practically_pass          0.0   
                                           trivial_failure          12.0   

best_prior                                                      Laplace  \
dataset_type   dataset    subset transform failure_category               
remote sensing agriVision full   fourier   actually_pass            0.0   
                                           interesting_failure      0.0   
                                           practically_pass         0.0   
                                 wavelet   actually_pass            0.0   
                                           practically_pass         0.0   
               pastis     full   fourier   actually_pass            0.0   
                                           practically_pass         0.0   
                                 learned   None                     0.0   
                                 wavelet   actually_pass            0.0   
                                           practically_pass         0.0   
               spaceNet   full   fourier   practically_pass         0.0   
                                 learned   None                     2.0   
                                 wavelet   actually_pass            0.0   
                                           practically_pass         0.0   
                                           trivial_failure          0.0   

best_prior                                                      Student-T  
dataset_type   dataset    subset transform failure_category   

# Natural Images

In [11]:
DATASET_TYPE = "natural" 

In [12]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,natural,coco,indoor,wavelet,diagonal,62.0,100.0
1,natural,coco,indoor,wavelet,horizontal,25.0,81.0
2,natural,coco,indoor,wavelet,vertical,84.0,94.0
3,natural,coco,outdoor,wavelet,diagonal,62.0,100.0
4,natural,coco,outdoor,wavelet,horizontal,25.0,84.0
5,natural,coco,outdoor,wavelet,vertical,88.0,100.0
6,natural,segmentAnything,full,learned,dual_color,0.0,100.0
7,natural,segmentAnything,full,learned,eye,0.0,100.0
8,natural,segmentAnything,full,learned,inside_out,0.0,44.0
9,natural,segmentAnything,full,learned,misc,0.0,67.0


In [13]:

medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                            actually_pass  \
dataset_type dataset         subset  transform orientation                  
natural      coco            indoor  wavelet   diagonal              38.0   
                                               horizontal             0.0   
                                               vertical              38.0   
                             outdoor wavelet   diagonal              38.0   
                                               horizontal             0.0   
                                               vertical              38.0   
             segmentAnything full    learned   dual_color             0.0   
                                               eye                    0.0   
                                               inside_out             0.0   
                                               misc                   0.0   
                                               multi_edge             0.0   
                                               single_edge            0.0   
                                     wavelet   diagonal              42.0   
                                               horizontal             0.0   
                                               vertical              33.0   

failure_category                                            practically_pass  \
dataset_type dataset         subset  transform orientation                     
natural      coco            indoor  wavelet   diagonal                 25.0   
                                               horizontal               25.0   
                                               vertical                 47.0   
                             outdoor wavelet   diagonal                 25.0   
                                               horizontal               25.0   
                                               vertical                 50.0   
             segmentAnything full    learned   dual_color                0.0   
                                               eye                       0.0   
                                               inside_out                0.0   
                                               misc                      0.0   
                                               multi_edge                0.0   
                                               single_edge               0.0   
                                     wavelet   diagonal                 58.0   
                                               horizontal               56.0   
                                               vertical                 67.0   

failure_category                                            interesting_failure  \
dataset_type dataset         subset  transform orientation                        
natural      coco            indoor  wavelet   diagonal                    38.0   
                                               horizontal                  12.0   
                                               vertical                    16.0   
                             outdoor wavelet   diagonal                    38.0   
                                               horizontal                  16.0   
                                               vertical                    12.0   
             segmentAnything full    learned   dual_color                   0.0   
                                               eye                          0.0   
                                               inside_out                   0.0   
                                               misc                         0.0   
                                               multi_edge                   0.0   
                                               single_edge                  0.0   
                                     wavelet   diagonal                     0.0   
                                               horizontal                   0.0   
 

In [14]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                  GenGamma  \
dataset_type dataset         subset  transform orientation             
natural      coco            indoor  wavelet   diagonal        100.0   
                                               horizontal       81.0   
                                               vertical         94.0   
                             outdoor wavelet   diagonal        100.0   
                                               horizontal       84.0   
                                               vertical        100.0   
             segmentAnything full    learned   dual_color      100.0   
                                               eye             100.0   
                                               inside_out       44.0   
                                               misc             67.0   
                                               multi_edge      100.0   
                                               single_edge     100.0   
                                     wavelet   diagonal        100.0   
                                               horizontal       86.0   
                                               vertical        100.0   

best_prior                                                  Gaussian  Laplace  \
dataset_type dataset         subset  transform orientation                      
natural      coco            indoor  wavelet   diagonal          0.0      0.0   
                                               horizontal       12.0      6.0   
                                               vertical          6.0      0.0   
                             outdoor wavelet   diagonal          0.0      0.0   
                                               horizontal        6.0      6.0   
                                               vertical          0.0      0.0   
             segmentAnything full    learned   dual_color        0.0      0.0   
                                               eye               0.0      0.0   
                                               inside_out       44.0     11.0   
                                               misc             17.0      0.0   
                                               multi_edge        0.0      0.0   
                                               single_edge       0.0      0.0   
                                     wavelet   diagonal          0.0      0.0   
                                               horizontal        3.0      8.0   
                                               vertical          0.0      0.0   

best_prior                                                  Student-T  
dataset_type dataset         subset  transform orientation             
natural      coco            indoor  wavelet   diagonal           0.0  
                                               horizontal         0.0  
                                               vertical           0.0  
                             outdoor wavelet   diagonal           0.0  
                                               horizontal         3.0  
                                               vertical           0.0  
             segmentAnything full    learned   dual_color         0.0  
                                               eye                0.0  
                                               inside_out         0.0  
                                               misc              17.0  
                                               multi_edge         0.0  
                                               single_edge        0.0  
                                     wavelet   diagonal           0.0  
                                               horizontal         3.0  
                                               vertical           0.0

In [15]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                          GenGamma  \
dataset_type dataset         subset  transform failure_category                
natural      coco            indoor  wavelet   actually_pass            92.0   
                                               interesting_failure     100.0   
                                               practically_pass        100.0   
                                               trivial_failure          70.0   
                             outdoor wavelet   actually_pass           100.0   
                                               interesting_failure     100.0   
                                               practically_pass        100.0   
                                               trivial_failure          74.0   
             segmentAnything full    learned   None                     89.0   
                                     wavelet   actually_pass           100.0   
                                               practically_pass        100.0   
                                               trivial_failure          69.0   

best_prior                                                          Gaussian  \
dataset_type dataset         subset  transform failure_category                
natural      coco            indoor  wavelet   actually_pass             8.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          20.0   
                             outdoor wavelet   actually_pass             0.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          11.0   
             segmentAnything full    learned   None                      8.0   
                                     wavelet   actually_pass             0.0   
                                               practically_pass          0.0   
                                               trivial_failure           6.0   

best_prior                                                          Laplace  \
dataset_type dataset         subset  transform failure_category               
natural      coco            indoor  wavelet   actually_pass            0.0   
                                               interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure         10.0   
                             outdoor wavelet   actually_pass            0.0   
                                               interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure         11.0   
             segmentAnything full    learned   None                     2.0   
                                     wavelet   actually_pass            0.0   
                                               practically_pass         0.0   
                                               trivial_failure         19.0   

best_prior                                                          Student-T  
dataset_type dataset         subset  transform failure_category                
natural      coco            indoor  wavelet   actually_pass              0.0  
                                               interesting_failure        0.0  
                                               practically_pass           0.0  
                                               trivial_failure            0.0  
                             outdoor wavelet   actually_pass              0.0  
                                               interesting_failure        0.0  
           

# Medical

In [16]:
DATASET_TYPE = "medical"

In [17]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,86.0,71.0
1,medical,syntheticMRI2D,axial,wavelet,horizontal,71.0,86.0
2,medical,syntheticMRI2D,axial,wavelet,vertical,71.0,71.0
3,medical,syntheticMRI2D,coronal,wavelet,diagonal,100.0,86.0
4,medical,syntheticMRI2D,coronal,wavelet,horizontal,71.0,71.0
5,medical,syntheticMRI2D,coronal,wavelet,vertical,71.0,71.0
6,medical,syntheticMRI2D,sagittal,wavelet,diagonal,71.0,71.0
7,medical,syntheticMRI2D,sagittal,wavelet,horizontal,71.0,86.0
8,medical,syntheticMRI2D,sagittal,wavelet,vertical,100.0,86.0
9,medical,syntheticMRI3D,full,wavelet,aad,67.0,67.0


In [18]:

medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                            actually_pass  \
dataset_type dataset        subset   transform orientation                  
medical      syntheticMRI2D axial    wavelet   diagonal               0.0   
                                               horizontal             0.0   
                                               vertical               0.0   
                            coronal  wavelet   diagonal              29.0   
                                               horizontal             0.0   
                                               vertical               0.0   
                            sagittal wavelet   diagonal               0.0   
                                               horizontal             0.0   
                                               vertical               0.0   
             syntheticMRI3D full     wavelet   aad                    0.0   
                                               ada                    0.0   
                                               add                    0.0   
                                               daa                    0.0   
                                               dad                   20.0   
                                               dda                    0.0   
                                               ddd                   40.0   

failure_category                                            practically_pass  \
dataset_type dataset        subset   transform orientation                     
medical      syntheticMRI2D axial    wavelet   diagonal                 86.0   
                                               horizontal               71.0   
                                               vertical                 71.0   
                            coronal  wavelet   diagonal                 71.0   
                                               horizontal               71.0   
                                               vertical                 71.0   
                            sagittal wavelet   diagonal                 71.0   
                                               horizontal               71.0   
                                               vertical                100.0   
             syntheticMRI3D full     wavelet   aad                      67.0   
                                               ada                      67.0   
                                               add                      80.0   
                                               daa                      60.0   
                                               dad                      60.0   
                                               dda                     100.0   
                                               ddd                      60.0   

failure_category                                            interesting_failure  \
dataset_type dataset        subset   transform orientation                        
medical      syntheticMRI2D axial    wavelet   diagonal                     0.0   
                                               horizontal                   0.0   
                                               vertical                     0.0   
                            coronal  wavelet   diagonal                     0.0   
                                               horizontal                  14.0   
                                               vertical                     0.0   
                            sagittal wavelet   diagonal                     0.0   
                                               horizontal                  14.0   
                                               vertical                     0.0   
             syntheticMRI3D full     wavelet   aad                         17.0   
                                               ada                         17.0   
                                               add                         20.0   
          

In [19]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                  GenGamma  \
dataset_type dataset        subset   transform orientation             
medical      syntheticMRI2D axial    wavelet   diagonal         62.0   
                                               horizontal       75.0   
                                               vertical         62.0   
                            coronal  wavelet   diagonal         75.0   
                                               horizontal       62.0   
                                               vertical         62.0   
                            sagittal wavelet   diagonal         62.0   
                                               horizontal       75.0   
                                               vertical         75.0   
             syntheticMRI3D full     wavelet   aad              57.0   
                                               ada              57.0   
                                               add              57.0   
                                               daa              57.0   
                                               dad              71.0   
                                               dda              86.0   
                                               ddd              86.0   

best_prior                                                  Gaussian  Laplace  \
dataset_type dataset        subset   transform orientation                      
medical      syntheticMRI2D axial    wavelet   diagonal         12.0     12.0   
                                               horizontal       25.0      0.0   
                                               vertical         38.0      0.0   
                            coronal  wavelet   diagonal          0.0      0.0   
                                               horizontal       25.0      0.0   
                                               vertical         25.0      0.0   
                            sagittal wavelet   diagonal         12.0      0.0   
                                               horizontal       12.0      0.0   
                                               vertical         12.0      0.0   
             syntheticMRI3D full     wavelet   aad              29.0      0.0   
                                               ada              14.0     14.0   
                                               add              14.0      0.0   
                                               daa              14.0     14.0   
                                               dad               0.0     14.0   
                                               dda               0.0      0.0   
                                               ddd               0.0      0.0   

best_prior                                                  Student-T  
dataset_type dataset        subset   transform orientation             
medical      syntheticMRI2D axial    wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
                            coronal  wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
                            sagittal wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
             syntheticMRI3D full     wavelet   aad                0.0  
                                               ada                0.0  
                                               add               14.0  
                                               daa                0.0  
                                               dad                0.0  
                                               dda                0.0  
                    

In [20]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                          GenGamma  \
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass         94.0   
                                               trivial_failure          12.0   
                            coronal  wavelet   actually_pass           100.0   
                                               interesting_failure       0.0   
                                               practically_pass         93.0   
                                               trivial_failure           0.0   
                            sagittal wavelet   interesting_failure     100.0   
                                               practically_pass         88.0   
                                               trivial_failure          17.0   
             syntheticMRI3D full     wavelet   actually_pass           100.0   
                                               interesting_failure       0.0   
                                               practically_pass        100.0   
                                               trivial_failure          25.0   

best_prior                                                          Gaussian  \
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass          0.0   
                                               trivial_failure          75.0   
                            coronal  wavelet   actually_pass             0.0   
                                               interesting_failure     100.0   
                                               practically_pass          0.0   
                                               trivial_failure          50.0   
                            sagittal wavelet   interesting_failure       0.0   
                                               practically_pass          6.0   
                                               trivial_failure          33.0   
             syntheticMRI3D full     wavelet   actually_pass             0.0   
                                               interesting_failure      25.0   
                                               practically_pass          0.0   
                                               trivial_failure          25.0   

best_prior                                                          Laplace  \
dataset_type dataset        subset   transform failure_category               
medical      syntheticMRI2D axial    wavelet   practically_pass         0.0   
                                               trivial_failure         12.0   
                            coronal  wavelet   actually_pass            0.0   
                                               interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure          0.0   
                            sagittal wavelet   interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure          0.0   
             syntheticMRI3D full     wavelet   actually_pass            0.0   
                                               interesting_failure     50.0   
                                               practically_pass         0.0   
                                               trivial_failure          6.0   

best_prior                                                          Student-T  
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass           6.0  
                                               trivial_failure            0.0  
                            coronal  wavelet   actually_pass              0.0  
            